# Parametric Poisson Equation with Adjoint Gradients

In this demo we consider the parametric Poisson's equation with homogeneous Dirichlet boundary conditions:

$$
\begin{align}
- \nu \Delta u &= f \quad \text{in } \Omega \\
u &= u_D \quad \text{on } \partial\Omega
\end{align}
$$

where $\Omega = [0,1]^2$ is the unit square, $\nu = 1$ is the diffusion coefficient and $f$ is a parameter.

We define the objective function (or quantity of interest):

$$
J(u) = \frac{1}{2}\int_\Omega \|u - g\|^2 \, dx
$$

where $g$ is a known function.


The goal of this demo is to compute the gradient of $J(u)$ with respect to $f$, $\nu$, and $u_D$ using the adjoint method.

We first start with the forward approach to solving the parametric Poisson's equation.
The weak formulation of the problem reads: find $u \in V$ such that

$$
a(u, v) = L(v) \quad \forall v \in V
$$

where

$$
\begin{align}
a(u, v) &= \int_\Omega \nu \nabla u \cdot \nabla v \, dx \\
L(v) &= \int_\Omega f v \, dx
\end{align}
$$

We solve this problem with a residual equation:

$$
F(u) = a(u, v) - L(v) = 0
$$

In [1]:
import dolfinx
import numpy as np
import scipy.sparse as sps
import ufl
from dolfinx import fem, mesh, nls
from mpi4py import MPI
from petsc4py.PETSc import ScalarType

from dolfinx_adjoint import *

We first need to create a graph object to store the computational graph.
This is done explicitly to maintain the guideline of FEniCSx.
Every function that is created with a graph object will be added to the graph
and its gradient will be computed automatically.

In [2]:
graph_ = Graph()
# Define mesh and finite element space
domain = mesh.create_unit_square(MPI.COMM_WORLD, 64, 64, mesh.CellType.triangle)
V = fem.functionspace(domain, ("CG", 1))
W = fem.functionspace(domain, ("DG", 0))

In [3]:
# Define the basis functions and parameters
uh = fem.Function(V, name="uₕ", graph=graph_)
v = ufl.TestFunction(V)
f = fem.Function(W, name="f", graph=graph_)
nu = fem.Constant(domain, ScalarType(1.0), name="ν", graph=graph_)

f.interpolate(lambda x: x[0] + x[1])

# Define the variational form and the residual equation
a = nu * ufl.inner(ufl.grad(uh), ufl.grad(v)) * ufl.dx
L = f * v * ufl.dx
F = a - L

# Define the boundary and the boundary conditions
domain.topology.create_connectivity(domain.topology.dim - 1, domain.topology.dim)
boundary_facets = mesh.exterior_facet_indices(domain.topology)
uD_L = fem.Function(V, name="u_D", graph=graph_)
uD_L.interpolate(lambda x: 1.0 + 0.0 * x[0])
uD_R = fem.Function(V, name="u_D")
uD_R.interpolate(lambda x: 1.0 + 0.0 * x[0])
uD_T = fem.Function(V, name="u_D")
uD_T.interpolate(lambda x: 1.0 + 0.0 * x[1])
uD_B = fem.Function(V, name="u_D")
uD_B.interpolate(lambda x: 1.0 + 0.0 * x[1])

boundary_dofs_L = fem.locate_dofs_geometrical(V, lambda x: np.isclose(x[0], 0.0))
boundary_dofs_R = fem.locate_dofs_geometrical(V, lambda x: np.isclose(x[0], 1.0))
boundary_dofs_T = fem.locate_dofs_geometrical(V, lambda x: np.isclose(x[1], 1.0))
boundary_dofs_B = fem.locate_dofs_geometrical(V, lambda x: np.isclose(x[1], 0.0))

# Store all boundary dofs in one array for testing
bc_dofs_total = np.concatenate(
    [boundary_dofs_L, boundary_dofs_R, boundary_dofs_T, boundary_dofs_B]
)

bcs = [
    fem.dirichletbc(uD_L, boundary_dofs_L, graph=graph_),
    fem.dirichletbc(uD_R, boundary_dofs_R),
    fem.dirichletbc(uD_T, boundary_dofs_T),
    fem.dirichletbc(uD_B, boundary_dofs_B),
]

# Define the problem solver and solve it
problem = fem.petsc.NewtonSolverNonlinearProblem(
    F,
    uh,
    bcs=bcs,
    graph=graph_,
)
solver = nls.petsc.NewtonSolver(MPI.COMM_WORLD, problem, graph=graph_)
solver.solve(uh, graph=graph_)
# Define profile g
g = fem.Function(W, name="g")
g.interpolate(
    lambda x: 1 / (2 * np.pi**2) * np.sin(np.pi * x[0]) * np.sin(np.pi * x[1])
)
# Define the objective function
alpha = fem.Constant(domain, ScalarType(1e-6), name="α")
J_form = 0.5 * ufl.inner(uh - g, uh - g) * ufl.dx + alpha * ufl.inner(f, f) * ufl.dx
J = fem.assemble_scalar(fem.form(J_form, graph=graph_), graph=graph_)

In [4]:
dJdf = graph_.backprop(id(J), id(f))
dJdnu = graph_.backprop(id(J), id(nu))
dJdbc = graph_.backprop(id(J), id(uD_L))
if MPI.COMM_WORLD.rank == 0:
    print("J(u) = ", J)
    print("dJdnu = ", dJdnu)
    print("||dJ/df||_L2 = ", np.sqrt(np.dot(dJdf, dJdf)))
    print("||dJ/dbc||_L2 = ", np.sqrt(np.dot(dJdbc, dJdbc)))

AttributeError: module 'dolfinx.la' has no attribute 'create_petsc_vector_wrap'